In [2]:
import random, shutil, os
import numpy as np
import pandas as pd

from mido import MidiFile
from glob import glob

In [6]:
def create_only_folders(dir, files):
    return [f for f in files if os.path.isfile(os.path.join(dir,f))]

shutil.copytree(r"C:\Users\KIST\Desktop\gmd\gmd", './bar_dataset', ignore=create_only_folders)

'./bar_dataset'

In [7]:
midi_list = glob(r"C:\Users\KIST\Desktop\gmd\gmd\\**\\**\\*.mid")

In [11]:
def kit_classify(note):
    cat = ''
    if note in [36]: # kick
        cat = 0
    elif note in [38, 40, 37]: # snare 
        cat = 1
    elif note in [48, 50]: # High Tom
        cat = 2
    elif note in [45, 47]: # Mid Tom
        cat = 3
    elif note in [43, 58]: # Low Tom
        cat = 4
    elif note in [46, 26]: # Open Hat
        cat = 5
    elif note in [42, 22, 44]: # Closed Hat
        cat = 6
    elif note in [49, 55, 57, 52]: # Crash
        cat = 7
    elif note in [51, 59, 53]: # Ride
        cat = 8

    return cat


# def kit_classify(note):
#     cat = ''
#     if note in [36]: # kick
#         cat = 0
#     elif note in [38, 40, 37]: # snare 
#         cat = 1
#     elif note in [48, 50]: # High Tom
#         # cat = 2
#         cat = 2 # Replace HT -> MT
#     elif note in [45, 47]: # Mid Tom
#         # cat = 3
#         cat = 2
#     elif note in [43, 58]: # Low Tom
#         # cat = 4
#         cat = 3
#     elif note in [46, 26]: # Open Hat
#         # cat = 5
#         cat = 4
#     elif note in [42, 22, 44]: # Closed Hat
#         # cat = 6
#         cat = 5
#     elif note in [49, 55, 57, 52]: # Crash
#         # cat = 7 
#         cat = 6 # Replace Cr -> Ri
#     elif note in [51, 59, 53]: # Ride
#         cat = 6
#     else:
#         print(note)

#     return cat

---

In [13]:
# mid2 = MidiFile('/Users/cooky/Desktop/groove/drummer7/session1/3_rock_86_fill_4-4.mid', clip=True)

for midi in midi_list:
    mid = MidiFile(midi, clip=True)
    on_sec = 0
    event = []

    for msg in mid.tracks[0]:
            note = str(msg).split()
            if len(note) == 5:
                on_sec += msg.dict()['time']

                if note[0] == 'note_on':
                    if [round(on_sec/120), msg.dict()['note']] not in event:
                        event.append([round(on_sec/120), msg.dict()['note']])
    
    beat = pd.DataFrame(0, index = np.arange(9), columns=np.arange( (event[-1][0]//16 + 1) * 16 ))

    for e in event:
        beat.loc[kit_classify(e[1]),e[0]] = 1

    if beat.shape[1] < 32:
        for i in range(beat.shape[1], 32):
            beat[i] = 0

    beat.to_csv('./bar_dataset'+midi[29:]+'.csv.gzip', index = False, compression='gzip')

---

In [55]:
csv_list = glob('./bar_dataset/**/**/*.mid.csv.gzip')

In [90]:
fill_cnt = 1
beat_cnt = 1

for csv in csv_list:
    imsi = pd.read_csv(csv, compression='gzip')
    if 'beat' in csv.lower():
        if imsi.shape[1] == 32:
            imsi.to_csv('./dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
            imsi[random.choices(imsi.columns, k=3)] = 0
            imsi.to_csv('./masked_dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
            beat_cnt += 1
        else:
            for i in range(imsi.shape[1] // 16 // 2):
                a = random.randrange(0, imsi.shape[1]-32)
                two_bars = imsi.iloc[:,a:a+32]
                two_bars.to_csv('./dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
                two_bars[random.choices(two_bars.columns, k=3)] = 0
                two_bars.to_csv('./masked_dataset/beat/'+str(beat_cnt)+'.csv.gzip', index=False, compression='gzip')
                beat_cnt += 1
    elif 'fill' in csv.lower():
        if imsi.shape[1] == 32:
            imsi.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
            imsi[random.choices(imsi.columns, k=3)] = 0
            imsi.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
            fill_cnt += 1
        else:
            for i in range(imsi.shape[1] // 16 // 2):
                a = random.randrange(0, imsi.shape[1]-32)
                two_bars = imsi.iloc[:,a:a+32]
                two_bars.to_csv('./dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
                two_bars[random.choices(two_bars.columns, k=3)] = 0
                two_bars.to_csv('./masked_dataset/fill/'+str(fill_cnt)+'.csv.gzip', index=False, compression='gzip')
                fill_cnt += 1
    

        



/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_bars[random.choices(two_bars.columns, k=3)] = 0
/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  two_bars[random.choices(two_bars.columns, k=3)] = 0
/var/folders/wn/fhcj6_9d2qb5d0pw_xw20ky80000gn/T/ipykernel_1498/545134225.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

---

In [2]:
imsi = pd.read_csv('./dataset/beat/1.csv.gzip', compression='gzip').to_numpy().swapaxes(1,0)

In [2]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0], # 0 null
    [1, 0, 0, 0, 0, 0, 0, 0, 0], # 1  B
    [1, 0, 0, 0, 0, 0, 0, 1, 0], # 2  B+C
    [1, 0, 0, 0, 0, 0, 1, 0, 0], # 3  B+CH
    [1, 0, 1, 0, 0, 0, 0, 0, 0], # 4  B+HT
    [1, 0, 0, 0, 1, 0, 0, 0, 0], # 5  B+LT
    [1, 0, 0, 1, 0, 0, 0, 0, 0], # 6  B+MT
    [1, 0, 0, 0, 0, 1, 0, 0, 0], # 7  B+OH
    [1, 0, 0, 0, 0, 0, 0, 0, 1], # 8  B+R
    [0, 0, 0, 0, 0, 0, 0, 1, 0], # 9  C
    [0, 0, 0, 0, 0, 0, 1, 0, 0], # 10 CH
    [0, 0, 1, 0, 0, 0, 1, 0, 0], # 11 CH+HT
    [0, 0, 0, 0, 1, 0, 1, 0, 0], # 12 CH+LT
    [0, 0, 0, 0, 0, 0, 1, 0, 1], # 13 CH+R
    [0, 0, 1, 0, 0, 0, 0, 0, 0], # 14 HT
    [0, 0, 1, 0, 1, 0, 0, 0, 0], # 15 HT+LT
    [0, 0, 0, 0, 1, 0, 0, 0, 0], # 16 LT
    [0, 0, 0, 1, 0, 0, 0, 0, 0], # 17 MT
    [0, 0, 0, 0, 0, 1, 0, 0, 0], # 18 OH
    [0, 0, 0, 0, 0, 0, 0, 0, 1], # 19 R
    [0, 1, 0, 0, 0, 0, 0, 0, 0], # 20 S
    [1, 1, 0, 0, 0, 0, 0, 0, 0], # 21 S+B
    [1, 1, 0, 0, 0, 0, 1, 0, 0], # 22 S+B+CH
    [1, 1, 0, 0, 1, 0, 0, 0, 0], # 23 S+B+LT
    [1, 1, 0, 0, 0, 1, 0, 0, 0], # 24 S+B+OH
    [1, 1, 0, 0, 0, 0, 0, 0, 1], # 25 S+B+R
    [0, 1, 0, 0, 0, 0, 0, 1, 0], # 26 S+C
    [0, 1, 0, 0, 0, 0, 1, 0, 0], # 27 S+CH
    [0, 1, 0, 0, 1, 0, 0, 0, 0], # 28 S+LT
    [0, 1, 0, 0, 0, 1, 0, 0, 0], # 29 S+OH
    [0, 1, 0, 0, 0, 0, 0, 0, 1], # 30 S+R
]


In [3]:

useful_list = []
true_count = 0

bars = glob('./dataset/beat/*.gzip')

for bar in bars: 
    useful = False
    cnt = 0
    imsi = pd.read_csv(bar, compression='gzip').to_numpy().swapaxes(1,0)

    for i in imsi:
        for ch in checking_arr:
            if np.array_equal(i, ch):
                cnt += 1
                break

    if cnt == 32:
        useful = True
        true_count += 1

    if useful:
        useful_list.append(bar)

# print(true_count)
# print(len(bars))


In [6]:
useful_list.__len__()

3556

In [20]:
file_cnt = 1
for file in useful_list:
    bar = pd.read_csv(file, compression='gzip').to_numpy().swapaxes(1,0)
    seq = []
    pd.DataFrame(bar).to_csv('./imsi/beat/' + str(file_cnt)+'.csv.gzip', index=False, compression='gzip')
    for note in bar:
        seq.append(checking_arr.index(note.tolist()))
        
        a = pd.DataFrame(np.zeros((32,23)))

        tmp = 0
        for i in seq:
            a.iloc[tmp, int(i)] = 1
            if tmp == 30:
                a.to_csv('./masked_single_label_dataset/beat/'+str(file_cnt)+'.csv.gzip', index=False, compression='gzip')
            tmp += 1
    # pd.DataFrame(seq).to_csv('./single_label_dataset/beat/'+str(file_cnt)+'.csv.gzip', index=False, compression='gzip')
    
    a.to_csv('./single_label_dataset/beat/'+str(file_cnt)+'.csv.gzip', index=False, compression='gzip')
    

    for i in range(9):
        bar[31][i] = 0
    
    pd.DataFrame(bar).to_csv('./imsi_mask/beat/' + str(file_cnt)+'.csv.gzip', index=False, compression='gzip')
    file_cnt += 1
    
    

In [15]:
print(pd.read_csv('single_label_dataset/beat/1.csv.gzip', compression='gzip'))

# a = pd.DataFrame(np.zeros((32,23)))


      0    1    2    3    4    5    6    7    8    9  ...   13   14   15   16  \
0   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
2   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
4   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
6   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
8   0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
10  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0  0.0   
11  0.0  0.0  0.0  0.0  0.0 

In [9]:
pd.read_csv('single_label_dataset/beat/1.csv.gzip', compression='gzip')

FileNotFoundError: [Errno 2] No such file or directory: 'single_label_dataset/beat/1.csv.gzip'